# 홍보 효과 분석
### 홍보 일자
- 11/ 19 홍보 포스터 부착
- 10 / 13 먼슬리 미팅
- 10 / 2 학보사 인터뷰 자료 출판
- 9/30 (2차 대규모 패치 - 세미콘 홍보)
- 9/3 (푸드라운지 식당 추가)
- 8/14 (1차 코드 리펙토링)
- 7/14 (지하철 블록 업데이트)
- 7/29 (막차 블록 업데이트)
- 5/23 (산돌이 꾸미기 - 프로필 사진 / 페르소나 등..)
- 5/21 (산돌팀 모집 및 1차 대규모 패치 홍보)
- 3/13 (산돌이 인스타그램 생성 및 홍보)
- 3/4 (신입생 홍보)
- 2/18 (에브리타임 산돌이 홍보?)

### 분석 개요
위 일자들에 대한 이용률 증가 및 효과가 좋았던 내용을 순위를 매겨
앞으로의 홍보 전략에 이바지하기 위함.

### 분석 방법
홍보일 이전 $$n$$일과, 이후 $$n$$일의 신규 사용자 및 활성사용자 수의 변화를 토대로  
홍보 효과를 분석.